In [5]:
#2019년부터 2007년까지 본문글에서 이전글 선택하여 넘기는 방법.
import json
from bs4 import BeautifulSoup
import html
import urllib.request
import requests
import time
from selenium import webdriver
import sqlite3
import datetime


con = sqlite3.connect("TheMinJoo.db")
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        date TEXT NOT NULL,
        content TEXT NOT NULL,
        url TEXT NOT NULL);
    """)
con.commit()

driver = webdriver.Chrome()
driver.get("http://theminjoo.kr/briefingDetail.do?bd_seq=2")


url = driver.current_url
html = requests.request("get",url)
soup = BeautifulSoup(html.text,'html')

for i in range(1, 7883):
    cur.execute("INSERT INTO table1(title,url,content,content_date)VALUES(?,?,?,?)",[
        driver.find_element_by_css_selector('.detail_view .title').text,
        driver.current_url,
        driver.find_element_by_css_selector('div.cont').text,
        driver.find_element_by_css_selector('span.txt').text])
    con.commit()
    cur.execute("SELECT * FROM table1")
    driver.find_element_by_css_selector('.other_view .nextview a').click()

driver.close()

SyntaxError: EOL while scanning string literal (<ipython-input-5-6d615c169910>, line 56)

In [ ]:
#년도 선택해서 해당 부분만 받아오기(2006년도)

con = sqlite3.connect("comment.db")
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        url TEXT NOT NULL);
        
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        url TEXT NOT NULL,
        content TEXT NOT NULL,
        content_date TEXT NOT NULL);
    """)
con.commit()

    

driver = webdriver.Chrome()
driver.get("http://theminjoo.kr/briefingList.do")
driver.find_element_by_xpath("//option[@value='2006']").click()
driver.find_element_by_css_selector('#listForm #container #content .srch_wrap.line1 a').click()

i = 0
while(i<157):  #5개 묶음 몇번돌지 정하기.
    
    for page in range(2,7):
        if(page == 6):
            page=1
            #5개단위로 넘기기.
            driver.find_element_by_xpath('//*[@id="content"]/div[3]/a[3]').click()
            i+=1
            
        li = driver.find_element_by_css_selector('.paging ul li a')   

        #각 안에 들어간 10개의 링크들
        for _ in driver.find_elements_by_css_selector(".bbs_list .tal > a"):
            if len(_.get_attribute("href"))>1:
                cur.execute("INSERT INTO table1(url) VALUES(?)",
                           [_.get_attribute("href"),])
                con.commit()
        cur.execute("SELECT * FROM table1")

        #1,2,3페이지. 
        driver.find_element_by_xpath('//*[@id="content"]/div[3]/ul[%s]/li/a' %page).click()
        

driver.execute_script("javascript:window.open('about:blank')")
driver.switch_to_window(driver.window_handles[-1])

for _ in  cur.fetchall():
    driver.get(_[1])
    cur.execute("INSERT INTO table2(title,url,content,content_date)VALUES(?,?,?,?)",[
        driver.find_element_by_css_selector('.detail_view .title').text,
        driver.current_url,
        driver.find_element_by_css_selector('div.cont').text,
        driver.find_element_by_css_selector('span.txt').text])
    con.commit()
    
driver.close()       